In [6]:
import time
tstart=time.clock()
print(tstart)
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans

In [3]:
def KMeansGen(raster, output, k):
        NDBI = np.ma.masked_outside(rasterio.open(raster).read(1),-1,1)
        MEI = np.ma.masked_outside(rasterio.open(raster).read(2),-1,1)
        VIGS = np.ma.masked_outside(rasterio.open(raster).read(3),-1,1)
        DBI = np.ma.masked_outside(rasterio.open(raster).read(4),-1,1)
        Qz_Calcite = np.ma.masked_outside(rasterio.open(raster).read(5),-1,1)
        VV = np.ma.masked_outside(rasterio.open(raster).read(6),-1,1)
        VH = np.ma.masked_outside(rasterio.open(raster).read(7),-1,1)
        
        data = pd.DataFrame({"NDBI":NDBI[NDBI==NDBI],"MEI":MEI[MEI==MEI], "VIGS":VIGS[VIGS==VIGS], "DBI":DBI[DBI==DBI], "Qz_Calcite":Qz_Calcite[Qz_Calcite==Qz_Calcite],"VV":VV[VV==VV], "VH":VH[VH==VH]})
        #for calculations using algorithms we have to drop NaN values. previous step!!!!
        data2= data.dropna()
        X = data2[["NDBI", "MEI", "VIGS", "DBI", "Qz_Calcite", "VV", "VH"]]
        kmeans= KMeans(n_clusters =k, random_state =0)
        Labels = kmeans.fit(X)
        #the array of the KMeans result does not contain the NaN values, so it is impossible to reshape to its original shape(raster). the nextstep is to find a way to add the Nan values (then nodata) to the labels. Maybe a for loop!!!
        Z= pd.DataFrame({"Labels":Labels.labels_})
        Z_Reindexed = Z.set_index(data2.index)
        data["Label"] = Z_Reindexed["Labels"]
        Result = pd.to_numeric(data["Label"], downcast = "float" )
        im = Result.values
        im = np.reshape(Result.values, (NDBI.shape[0], NDBI.shape[1] ))
        ##### After, Save
        KMean_output = rasterio.open(output, "w", driver = "GTiff", height = NDBI.shape[0], width = NDBI.shape[1], dtype = NDBI.dtype, count = 1, nodata = rasterio.open(raster).nodata, crs = "+proj = utm", transform = rasterio.open(raster).transform)
        KMean_output.write(im, 1)
        KMean_output.close()

### Run the code below, and open the image from the output folder in arcGIS Pro (arcmap does not open properly (32bit!!))

In [5]:
raster= "/Users/arngolo/Downloads/RESEARCH_Angolo/PYTHON/NDBI_MEI_VIGS_DBI_QzCa_VV_VH_10m.tif"
output="/Users/arngolo/Downloads/RESEARCH_Angolo/PYTHON/Program_Result/Luanda_NDBI_MEI_VIGS_DBI_QzCa_VV_VH_10m_k8.tif"
k=8
KMeansGen(raster, output, k)
tend=time.clock()
print(tend)

### change the coordinate system accordingly!!